# Cost Analysis Baseline - 2018 Over 80 Cases by Case Origin

Cases classified as 'Ignored Cases' have been removed in this analysis.  Adjudicated cases have not been removed in order to include this feature in some analysis.

## Description

In [ ]:
#hide
#hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#hide
import pandas as pd
import numpy  as np
from datetime import datetime
import altair as alt
_=alt.data_transformers.disable_max_rows()
#year_portion_limit = 0.8
debug = 0
from cheeky_monkey.core import *
from cheeky_monkey.core.baseline import *

data_file_path = r'C:\Users\mbeaulieu\rsi_project_book\Data\\'


## 2018 Team Capacity as expressed by Full Time Employee Days (FTE Days)

Figure 1.  FTE (Days) for each month in 2018.  This is a representation of the team's capacity, and as such, it will be used for deriving a baseline rate of work.  The work will be derived from the number of status changes in cases.

In [ ]:
cases2018_df = get_processed_case_data(data_file_path)

In [ ]:
#hide
aggregation = {
    'Case Count': ('DRIVERS_LICENSE_NO','count'),
    'Total Status Count': ('STATUS_COUNT','sum'),
    'Avg Case Length': ('case_length_days','mean'),
    'Avg Turnaround Time': ('turnaround_time_days','mean')

}


case_agecategory_summary_data = cases2018_df[['ORIGIN_DSC', 'DRIVERS_LICENSE_NO','STATUS_COUNT', 'case_length_days', 'turnaround_time_days' ]] \
    .groupby(['ORIGIN_DSC'], as_index=False).agg(** aggregation)

case_agecategory_summary_data.columns = ['Case Origin',   'Case Count', 'Status Change Count','Average Case Length', 'Average Turnaround Time']

case_agecategory_summary_data['Status Changes per Case'] = case_agecategory_summary_data.apply( lambda x: x['Status Change Count'] / x['Case Count'], axis=1)

#print(case_agecategory_summary_data['Status Change Count'].sum())
#case_agecategory_summary_data


In [ ]:
#hide
aggregation = {
    'Cases': ('STATUS_COUNT','size'),
    'Total Status Count': ('STATUS_COUNT','sum')
}



case_summary_data = cases2018_df.groupby(['ORIGIN_DSC', 'STATUS_COUNT']) \
.agg(** aggregation).reset_index()



case_summary_data.columns = ['Origin Report',  'Status Count', 'Cases', 'Total Status Count']

case_summary_data['Total Cases in Group'] = case_summary_data.groupby(['Origin Report'])['Cases'].transform(lambda x: sum(x) )
case_summary_data['Total Status Changes in Group'] = case_summary_data.groupby(['Origin Report'])['Total Status Count'].transform(lambda x: sum(x) )
case_summary_data['% Cases'] = case_summary_data.groupby(['Origin Report'])['Cases'].apply(lambda x: 100 * x / x.sum() )
case_summary_data['% Status Change Count'] = case_summary_data.groupby(['Origin Report'])['Total Status Count'].apply(lambda x: 100 * x / x.sum() )

case_summary_data['Total Status Changes in Year'] =  case_summary_data['Total Status Count'].sum() 
case_summary_data['Total Cases in Year'] =  case_summary_data['Cases'].sum() 
case_summary_data['% Group Cases'] = case_summary_data.apply(lambda x: 100 * ( x['Total Cases in Group'] / x['Total Cases in Year'] ) , axis=1)
case_summary_data['Status Change per FTE Day'] =  case_summary_data['Total Status Count'].sum() / (12 * 400) 

case_summary_data['Status Changes/Case'] = case_summary_data.apply(lambda x: ( x['Total Status Count'] / x['Cases'] ) , axis=1)
case_summary_data['Group Status Changes/Case'] = case_summary_data.apply(lambda x: ( x['Total Status Changes in Group'] / x['Total Cases in Group'] ) , axis=1)
case_summary_data['Avg Yearly Status Changes/Case'] = case_summary_data.apply(lambda x: ( x['Total Status Changes in Year'] / x['Total Cases in Year'] ) , axis=1)


# case_summary_data['Group FTE Day/Case'] = case_summary_data.apply(lambda x: (  .011 * x['Group Status Changes/Case']  ) , axis=1)
# case_summary_data['Avg FTE Day/Case'] = case_summary_data.apply(lambda x: (  .011 * x['Avg Yearly Status Changes/Case']  ) , axis=1)
# case_summary_data['Group FTE Days'] = case_summary_data.apply(lambda x: (  .011 * x['Total Status Changes in Group']  ) , axis=1)



The goal with this analysis is to break down the 2018 cases by the Over 80 age group as well as Case origin.

FTE Days for each month in 2018 has been provided by Dave.

With the use of FTE Days, we can relate the team resources to the work performed, as estimated by the number of *Status Changes* in total.  Case duration is also analyzed, but since this measure is dependent on the responsiveness of the BC Driver, the Status Change measure has been chosen.

### Data Preparation

1. Select Cases from 2018
2. Filter out Ignored Cases
        a. ORIGIN_CD (9,17)
        b. CASE_CD (9,10)
        c. STATUS_COUNT 1 AND DECISION_CD 21
        d. OPENED_YEAR > 2022
        e. Missing Drivers Licence
        f. Missing Birthdate
3. Filter out Non-Adjudicated cases.  When the Case second Status is one of the closed status values
4. Create new dimensions 
        a. Origin & Decision
        b. First Status & Second Status





In [ ]:
##hide_input


loadedstatus_df = get_statuses(data_file_path)
s = loadedstatus_df[['STATUS_CD','STATUS_DSC']].reset_index(drop=True)
t = s[ (s['STATUS_CD'] == 4)  | (s['STATUS_CD'] == 5) | (s['STATUS_CD'] == 8) | (s['STATUS_CD'] == 9)  | (s['STATUS_CD'] == 14) | (s['STATUS_CD'] == 31) | (s['STATUS_CD'] == 32)  | (s['STATUS_CD'] == 34) | (s['STATUS_CD'] == 43)  ][['STATUS_CD','STATUS_DSC']].reset_index(drop=True)
t


,STATUS_CD,STATUS_DSC
0,4,CLOSED - DECISION MADE
1,5,CLOSED - DECISION MADE/SYSTEM CLOSED
2,8,CLOSED - AUTO
3,9,CLOSED - FOLLOWUP
4,14,CLOSED - NON COMPLY EXAM
5,31,CLOSED MEDICAL - OPEN REHAB
6,32,CLOSED - ROUTINE FOLLOW UP
7,34,CLOSED NON-COMPLY RDP OR INTERLOCK
8,43,CLOSED - CLEANUP PROJECT


## Looking at First and Second Actions on an Adjudicate Case


In [ ]:
#hide

cols = ['ORIGIN_DSC','Origin Report', 'DECISION_DSC','Origin & Decision','OUTCOME_GROUP_REPORT','STATUS_COUNT', 'STATUS_HISTORY_x','COUNT_ERA_CONSULTATIONS','COUNT_PENDING_ERA','COUNT_ERA_REGULAR_REVIEW', 'COUNT_ERA_IMMEDIATE_REVIEW','DOCUMENT_COUNT']

#cases2018_df[cols]

In [ ]:
#hide

aggregation = {
    'Case Count': ('STATUS_COUNT','size'),
#    'Case Count Test': ('DRIVERS_LICENSE_NO','nunique'),
    'Total Status Change Count': ('STATUS_COUNT','sum')
}

case_summary_data_by_status_type = cases2018_df.groupby(['ORIGIN_DSC', 'DECISION_DSC']) \
.agg(** aggregation).reset_index()


case_summary_data_by_status_type['Cumulative Cases'] = case_summary_data_by_status_type['Case Count'].cumsum()
case_summary_data_by_status_type['Cumulative Counts'] = case_summary_data_by_status_type['Total Status Change Count'].cumsum()

# print(f"Total Case Count: {case_summary_data_by_status_type['Case Count'].sum():,.0f}" )
# print(f"Total Status Change Count: {case_summary_data_by_status_type['Total Status Change Count'].sum():,.0f}" )


# print(f"Total Case Count - Adjudicated: {case_summary_data_by_status_type['Case Count'].sum():,.0f}" )
# print(f"Total Status Change Count - Adjudicated: {case_summary_data_by_status_type['Total Status Change Count'].sum():,.0f}" )

#case_summary_data_by_status_type


In [ ]:
##hide_input


aggregation = {
    'Case Count': ('STATUS_COUNT','size'),
    'Total Status Change Count': ('STATUS_COUNT','sum')
}
case_summary_data_by_status_type = cases2018_df.groupby(['Is ERA Dsc','Initial Status Desc']) \
.agg(** aggregation).reset_index()

print(f"Total Case Count: {case_summary_data_by_status_type['Case Count'].sum():,.0f}" )
print(f"Total Status Change Count: {case_summary_data_by_status_type['Total Status Change Count'].sum():,.0f}" )

bar1 = alt.Chart(case_summary_data_by_status_type).mark_bar().encode(
    alt.X('sum(Case Count):Q', title="Total Cases"),
#    y= 'Initial Status Desc',
    y= alt.Y('Initial Status Desc',
        sort=alt.EncodingSortField(
            field="Case Count", op="sum",
            order="descending"
            )
        ),

    tooltip=['sum(Case Count):Q'],
    color='Is ERA Dsc'
)

bar1


Total Case Count: 10,745
Total Status Change Count: 40,054


alt.Chart(...)

Figure 1. Frequency distribution of cases by the First Status of the Case and by ERA.

In [ ]:
#hide
aggregation = {
    'Case Count': ('STATUS_COUNT','size'),
    'Total Status Change Count': ('STATUS_COUNT','sum')
}
case_summary_data_by_status_type = cases2018_df.groupby(['Is ERA Dsc', 'Second Status Desc']) \
.agg(** aggregation).reset_index()

print(f"Total Case Count: {case_summary_data_by_status_type['Case Count'].sum():,.0f}" )
print(f"Total Status Change Count: {case_summary_data_by_status_type['Total Status Change Count'].sum():,.0f}" )

bar1 = alt.Chart(case_summary_data_by_status_type).mark_bar().encode(
    alt.X('sum(Case Count):Q', title="Total Cases"),
    #y= 'Second Status Desc',
    y= alt.Y('Second Status Desc',
            sort=alt.EncodingSortField(
                field="Case Count", op="sum",
                order="descending"
                )
            ),
    color='Is ERA Dsc', 
    #x = alt.X('Case Count', aggregate='sum', type='quantitative'),
    tooltip=['sum(Case Count):Q'],
    #row='SECOND_CASE_STATUS_CD:N'
)

bar1

Total Case Count: 10,745
Total Status Change Count: 40,054


alt.Chart(...)

Figure 2.  Frequency distribution of cases by the Second Status of the Case, and by ERA.

## Looking at cases grouped by First and Second Statuses combined

In [ ]:
#hide

aggregation = {
    'Case Count': ('STATUS_COUNT','size'),
    'Total Status Change Count': ('STATUS_COUNT','sum')
}
case_summary_data_by_status_type = cases2018_df.groupby(['Is ERA Dsc', 'First & Second Status Descriptions']) \
.agg(** aggregation).reset_index()

print(f"Total Case Count: {case_summary_data_by_status_type['Case Count'].sum():,.0f}" )
print(f"Total Status Change Count: {case_summary_data_by_status_type['Total Status Change Count'].sum():,.0f}" )

bar1 = alt.Chart(case_summary_data_by_status_type).mark_bar().encode(
    alt.X('sum(Case Count):Q', title="Total Cases"),
    y= alt.Y('First & Second Status Descriptions',
            sort=alt.EncodingSortField(
                field="Case Count", op="sum",
                order="descending"
                )
            ),
    color='Is ERA Dsc', 
    #x = alt.X('Case Count', aggregate='sum', type='quantitative'),
    tooltip=['First & Second Status Descriptions', 'sum(Case Count):Q'],
    #row='SECOND_CASE_STATUS_CD:N'
)
bar1



Total Case Count: 10,745
Total Status Change Count: 40,054


alt.Chart(...)

Figure 3.  A new feature has been added to the dataset to classify cases.  It is a concatenation of the first and second statuses of the case.  This is the frequency distribution of these combinations, including ERA.

In [ ]:
#hide
aggregation = {
    'Case Count': ('STATUS_COUNT','size'),
    'Total Status Change Count': ('STATUS_COUNT','sum')
}
case_summary_data_by_case_type = cases2018_df.groupby(['ORIGIN_DSC']) \
.agg(** aggregation).reset_index()
print(f"Total Case Count: {case_summary_data_by_case_type['Case Count'].sum():,.0f}" )
print(f"Total Status Change Count: {case_summary_data_by_case_type['Total Status Change Count'].sum():,.0f}" )
bar1 = alt.Chart(case_summary_data_by_case_type).mark_bar().encode(
    alt.X('sum(Case Count):Q', title="Total Cases"),
    #alt.X('Age Category'),
    #x = alt.X('Case Count', aggregate='sum', type='quantitative'),
    tooltip=['sum(Case Count):Q'],
    color='ORIGIN_DSC'

)
bar2 = alt.Chart(case_summary_data_by_case_type).mark_bar().encode(
    #alt.X(alt.X('Age Category'),
    #x = alt.X('Total Status Change Count', aggregate='sum', type='quantitative', title="Total Status Changes"),
    x = alt.X('sum(Total Status Change Count):Q', title="Total Status Changes"),
    tooltip=['sum(Total Status Change Count):Q'],
    color='ORIGIN_DSC'
)
(bar1 & bar2) #.properties( width = 500 )

Total Case Count: 10,745
Total Status Change Count: 40,054


alt.VConcatChart(...)

Figure 4. Total Cases and Total Status Changes (2018), by Origin

In [ ]:
#hide

# total_status_change_count = case_closedunder12_agecategory_summary_data['Status Change Count'].sum()
# _2018_status_change_count = case_closedunder12_agecategory_summary_data['Total Status Proportion'].sum()
# _2018_yearly_completion_rate = _2018_status_change_count / total_status_change_count
# print(f"Total Case Count: {case_closedunder12_agecategory_summary_data['Case Count'].sum():,.0f}" )
# print(f"Total Status Change Count: {case_closedunder12_agecategory_summary_data['Status Change Count'].sum():,.0f}" )

# #print(f"2018 Portion of Status Change Count: {_2018_status_change_count:.2f}" )
# #print(f"2018 Completion Rate:  {_2018_yearly_completion_rate :.2%} ")
# bar1 = alt.Chart(case_closedunder12_agecategory_summary_data).mark_bar().encode(
#     alt.X('sum(Case Count):Q', title="Total Cases"),
#     #alt.X('Age Category'),
#     #x = alt.X('Case Count', aggregate='sum', type='quantitative'),
#     tooltip=['sum(Case Count):Q'],
# )
# bar2 = alt.Chart(case_closedunder12_agecategory_summary_data).mark_bar().encode(
#     #alt.X(alt.X('Age Category'),
#     x = alt.X('Total Status Proportion', aggregate='sum', type='quantitative', title="Total Status Changes"),
#     tooltip=['sum(Total Status Proportion):Q'],
# )
# bar1 & bar2

In [ ]:
#hide
open_closed_summary = cases2018_df.groupby(['Is ERA Dsc','STATUS_COUNT'])[['DRIVERS_LICENSE_NO']].count().reset_index()
open_closed_summary.columns = ['Is ERA Dsc','Case Status Count', 'Count']
bar1 = alt.Chart(open_closed_summary).mark_bar().encode(
    
    x= alt.X('Count:Q', title="Total Cases"),
    y= alt.Y('Case Status Count:N', title="Case Status Count"),
    #alt.X('Age Category'),
    #x = alt.X('Case Count', aggregate='sum', type='quantitative'),
    tooltip=['Count:Q'],
    color='Is ERA Dsc:N'
)
bar1

alt.Chart(...)

Figure 5: Cases categorized by the number of Case Status Counts

## Analysis of Cases 

### Deriving the baseline case activity, represented by Status Changes/Case

The current dataset includes the number of Status Changes that were made for each case.  This section shows the breakdown of cases by number of Status Changes that were made in each case. 

:::{note}
The dataset was downloaded October 1, 2021.
The dataset value for the Status Changes of each case have been reduced by 1, to eliminate the initial status of a Case. 
However the initial status of the case is still used in analysis.
:::

In [ ]:
#hide

base =  alt.Chart(case_summary_data).mark_bar().encode(
    alt.X('sum(Cases):Q', title="Total Cases"),
    color = alt.Color('Status Changes/Case:N'),
    #row='Age Category'
    #opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)
bar1 = base.encode(
    alt.Y('Origin Report:N'),
    alt.X('sum(Cases):Q', title="Total Cases"),
    tooltip=['Status Changes/Case', 'Cases', 'mean(Total Status Changes in Group)', 'mean(Total Cases in Group)', 'mean(% Group Cases)'],
    color = alt.Color('Status Changes/Case:N'),
    #row='Age Category'
    #opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
)
# bar2 = base.encode(
#     alt.Y('Age Category:N'),
#     alt.X('sum(Cases):Q', title="Total Cases"),
#     tooltip=['Status Changes/Case', 'Cases', 'mean(Total Status Changes in Group)', 'mean(Total Cases in Group)', 'mean(% Group Cases)'],
#     color = alt.Color('Status Changes/Case:N'),
#     #row='Age Category'
#     #opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
# )

base & bar1 # & bar2
# .properties(
#     width=500
# )



alt.VConcatChart(...)

Figure 6. 2018 Cases broken down by the frequency of cases with a certain Status Change Count and by Origin

In [ ]:
#hide

base =  alt.Chart(case_summary_data).mark_bar().encode(
    x = alt.X('Status Count:O', title="Status Count per Case"),
    y = alt.Y('Cases:Q', title="Total Cases"),
#    color = alt.Color('Status Changes/Case:N'),
    row='Origin Report'
    #opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).properties( height=100)


# bar1 = base.encode(
#     alt.Y('Case Type:N'),
#     alt.X('sum(Cases):Q', title="Total Cases"),
#     tooltip=['Status Changes/Case', 'Cases', 'mean(Total Status Changes in Group)', 'mean(Total Cases in Group)', 'mean(% Group Cases)'],
#     color = alt.Color('Status Changes/Case:N'),
#     #row='Age Category'
#     #opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
# )

base

alt.Chart(...)

Figure 7. Case Type Total Cases broken down by Status Count/Case

In [ ]:
#hide

aggregation = {
    'Cases': ('STATUS_COUNT','size'),
    'Total Status Count': ('STATUS_COUNT','sum'),
    'Total Case Length for All Cases': ('case_length_days', 'sum'),
    'Average Case Length in Days': ('case_length_days', 'mean')
}

#daily_case_summary_data = cases2018_df.groupby(['STATUS_COUNT', 'Type & Origin Description', 'GENERAL_STATUS','Case Length Over 30 Days', 'Case Length Over 60 Days']) \

daily_case_summary_data = cases2018_df.groupby(['STATUS_COUNT', 'Origin & Decision', 'GENERAL_STATUS']) \
.agg(** aggregation).reset_index()



In [ ]:
#hide

selection = alt.selection_multi(fields=['Origin & Decision']) #, bind='legend')

base = alt.Chart(daily_case_summary_data)

xscale = alt.Scale(domain=(1.0, 1000))
yscale = alt.Scale(domain=(1.9, 4.55))

area_args = {'opacity': .05, 'interpolate': 'step'}
#area_args = {'interpolate': 'step'}

points = base.mark_point().encode(
    alt.X('Average Case Length in Days:Q'), #,  scale=xscale),
    alt.Y('STATUS_COUNT', type='quantitative'), #, scale=yscale),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05)),
    #color='Type & Origin Desc',
    color=alt.Color('Origin & Decision',
            sort=alt.EncodingSortField('count(STATUS_COUNT)',  order='descending')),
    tooltip='Cases'
#)
# .add_selection(
#     selection
).interactive()

bottom_hist = base.mark_bar().encode(
    alt.X('Origin & Decision:N',
          # when using bins, the axis scale is set through
          # the bin extent, so we do not specify the scale here
          # (which would be ignored anyway)
          #bin=alt.Bin(maxbins=20 ), # , extent=xscale.domain),
          stack=None,
          title='', sort='-y',
         ),
    alt.Y('sum(Cases)', stack=None, title=''),
    alt.Color('Origin & Decision:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01)),


).properties(height=60)

top_hist = base.mark_area(**area_args).encode(
    alt.X('Average Case Length in Days:Q',
          # when using bins, the axis scale is set through
          # the bin extent, so we do not specify the scale here
          # (which would be ignored anyway)
          bin=alt.Bin(maxbins=20 ), # , extent=xscale.domain),
          stack=None,
          title=''
         ),
    alt.Y('sum(Cases)', stack=None, title=''),
    alt.Color('Origin & Decision:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01))

).properties(height=60)

right_hist = base.mark_area(**area_args).encode(
    alt.Y('STATUS_COUNT:Q',
          bin=alt.Bin(maxbins=20), #, extent=yscale.domain),
          stack=None,
          title='',
         ),
    alt.X('sum(Cases)', stack=None, title=''),
    alt.Color('Origin & Decision:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01))
).properties(width=60)

color = alt.condition(selection,
                      alt.Color('Origin & Decision:O', legend=None, 
                      scale=alt.Scale(scheme='category10')),
                      alt.value('lightgray'))

legend = base.mark_point(filled=True, size=200
).encode(
    y=alt.Y('Origin & Decision:O', axis=alt.Axis(orient='right')),
    color=color,
    size='sum(Cases)'
).add_selection(
selection
)

legend | (top_hist & (points | right_hist) & bottom_hist ).add_selection(
    selection
)
    

alt.HConcatChart(...)

Figure 8.  Interactive visualization.  Case Type and Origin have been combined to create another categorization.

In [ ]:
# #selection = alt.selection_multi(fields=['Type & Origin Description']) #, bind='legend')
# brush = alt.selection(type='interval', encodings=['x','y'])

# # alt.Chart(source).mark_point().encode(
# #     x='Horsepower:Q',
# #     y='Miles_per_Gallon:Q',
# #     color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
# # ).add_selection(brush)

# base = alt.Chart(daily_case_summary_data)

# xscale = alt.Scale(domain=(1.0, 1000))
# yscale = alt.Scale(domain=(1.9, 4.55))

# area_args = {'opacity': .05, 'interpolate': 'step'}
# #area_args = {'interpolate': 'step'}

# points = base.mark_point().encode(
#     alt.X('Average Case Length in Days:Q'), #,  scale=xscale),
#     alt.Y('STATUS_COUNT', type='quantitative'), #, scale=yscale),
#     opacity=alt.condition(brush, alt.value(1), alt.value(0.05)),
#     #color='Type & Origin Desc',
#     color=alt.Color('Type & Origin Description',
#             sort=alt.EncodingSortField('count(STATUS_COUNT)',  order='descending'))
# )
# #.add_selection(brush)

# bottom_hist = base.mark_bar().encode(
#     alt.X('STATUS_COUNT:N',
#           # when using bins, the axis scale is set through
#           # the bin extent, so we do not specify the scale here
#           # (which would be ignored anyway)
#           #bin=alt.Bin(maxbins=20 ), # , extent=xscale.domain),
#           stack=None,
#           title='', sort='-y',
#          ),
#     alt.Y('count()', stack=None, title=''),
# #    alt.Color('Type & Origin Description:N'),
#     opacity=alt.condition(brush, alt.value(1), alt.value(0.01)),
# ).transform_filter(
#     brush
# ).properties(height=60)

# # top_hist = base.mark_area(**area_args).encode(
# #     alt.X('Average Case Length in Days:Q',
# #           # when using bins, the axis scale is set through
# #           # the bin extent, so we do not specify the scale here
# #           # (which would be ignored anyway)
# #           bin=alt.Bin(maxbins=20 ), # , extent=xscale.domain),
# #           stack=None,
# #           title=''
# #          ),
# #     alt.Y('count()', stack=None, title=''),
# #     alt.Color('Type & Origin Description:N'),
# #     opacity=alt.condition(selection, alt.value(1), alt.value(0.01))

# # ).properties(height=60)

# # right_hist = base.mark_area(**area_args).encode(
# #     alt.Y('STATUS_COUNT:Q',
# #           bin=alt.Bin(maxbins=20), #, extent=yscale.domain),
# #           stack=None,
# #           title='',
# #          ),
# #     alt.X('count()', stack=None, title=''),
# #     alt.Color('Type & Origin Description:N'),
# #     opacity=alt.condition(selection, alt.value(1), alt.value(0.01))
# # ).properties(width=60)

# # color = alt.condition(selection,
# #                       alt.Color('Type & Origin Description:O', legend=None, 
# #                       scale=alt.Scale(scheme='category10')),
# #                       alt.value('lightgray'))

# # legend = base.mark_point(filled=True, size=200
# # ).encode(
# #     y=alt.Y('Type & Origin Description:O', axis=alt.Axis(orient='right')),
# #     color=color,
# #     size='count()'
# # ).add_selection(
# # selection
# # )

# #points

# (points & bottom_hist ).add_selection(
#     brush
# )
   
    

## Team Effort for Different Case Types


The total number of FTE Days in the year is estimated to be 12 times 400 FTE Days/Month = 4800 FTE Days/Year

#### 2019 What was the Team Effort for different Case Types?

* by frequency of status counts within each case
* Work Effort will be calculated in FTE Days
* Cases have a number of status changes, so we'll use number of status changes in each case to calculate the Work Effort for each case, using FTE Days (FTE Day)

In [ ]:
#hide

FTE_DAYS = 4800
FTE_DAYS_MONTH = 400

# aggregation = {
#     'Case Count': ('STATUS_COUNT','size'),
# #    'Case Count2': ('DRIVERS_LICENSE_NO','size'),
    
#     'Total Status Change Count': ('STATUS_COUNT','sum')
# }



# df = cases2018_df.groupby(['ORIGIN_DSC']) \
# .agg(** aggregation).reset_index()

# df['Group Status Change/Case'] = df.apply(lambda x: x['Total Status Change Count']/x['Case Count']  , axis=1)

# #df = df[df['Is Adjudicated'] == 'Adjudicated']
# case_count = df['Case Count'].sum()
# status_change_count = df['Total Status Change Count'].sum()

# #print(f"2018 Portion of Status Change Count: {_2018_status_change_count:.2f}" )

# print(f"Adjudicated Case Count for 2018: {case_count:,}")
# print(f"Adjudicated Status Change Count for 2018: {status_change_count:,}" )
# print(f"Average Status Change Count/Case: {status_change_count/case_count : .2f}" )

# print(f"Adjudicated Cases/FTE Day: {case_count/FTE_DAYS : .2f}") 
# print(f"Status Changes/FTE Day: { status_change_count/FTE_DAYS: .2f}")
# print(f"Monthly Team Capacity (Status Changes) { (status_change_count/FTE_DAYS) * FTE_DAYS_MONTH: ,.2f}" )


# ftedays_case_count = FTE_DAYS/case_count
# ftedays_status_change_count = FTE_DAYS/status_change_count
# print(f"FTE Days/Adjudicated Case: {ftedays_case_count : .3f}")
# print(f"FTE Days/Status Change {ftedays_status_change_count: .3f}") 

In [ ]:
#hide

aggregation = {
    'Case Count': ('DRIVERS_LICENSE_NO','size'),
    'Status Count': ('STATUS_COUNT','sum'),
    }

Case2018_monthly_counts = cases2018_df.groupby([pd.Grouper(freq='M', key='CASE_OPENED_DT') , 
                              pd.Grouper(key='ORIGIN_DSC'),                                   
                             ]).agg(** aggregation)

Case2018_monthly_counts = pd.DataFrame(Case2018_monthly_counts).reset_index()
Case2018_monthly_counts['Opened Month'] = Case2018_monthly_counts.apply(lambda x: x['CASE_OPENED_DT'].strftime('%b') + '-' + x['CASE_OPENED_DT'].strftime('%Y'), axis=1)

#Case2018_monthly_counts

In [ ]:
#hide

#df = pd.merge(Case2018_monthly_counts, ftedays_df, how='left',  left_on='Opened Month', right_on='Month-Year')


In [ ]:
#hide

#%load ./helpers.py

# %run ./helpers.py
# r = derive_statuscount_percase(cases2018_df, ftedays_df)

ftedays_df = get_fte_days_2018()
r = derive_statuscount_percase(cases2018_df, ftedays_df)
monthly_summary_statuscounts = r[0]


FTE_DAYS_YEAR 4154
df shape  (84, 11)
Adjudicated Case Count for 2018: 10,745
Adjudicated Status Change Count for 2018: 40,054
Average Status Change Count/Case:  3.73
Adjudicated Cases/FTE Day:  2.59
Status Changes/FTE Day:  9.64
FTE Days/Adjudicated Case:  0.387
FTE Days/Status Change  0.104


In [ ]:
#hide

ftedays_case_count = r[1]

ftedays_status_change_count = r[2]

# case_summary_data['Group FTE Days/Case'] = case_summary_data.apply(lambda x: (  ftedays_status_change_count * x['Group Status Changes/Case']  ) , axis=1)
# case_summary_data['Avg FTE Days/Case'] = case_summary_data.apply(lambda x: (  ftedays_status_change_count * x['Avg Yearly Status Changes/Case']  ) , axis=1)
# case_summary_data['Group FTE Days'] = case_summary_data.apply(lambda x: (  ftedays_status_change_count * x['Total Status Changes in Group']  ) , axis=1)

monthly_summary_statuscounts['Group Case Count/FTE'] = monthly_summary_statuscounts.apply(lambda x: x['Case Count']/x['FTE (Days)']  , axis=1)
monthly_summary_statuscounts['Group Status Change/FTE'] = monthly_summary_statuscounts.apply(lambda x: x['Total Status Change Count']/x['FTE (Days)']  , axis=1)

monthly_summary_statuscounts['Group FTE Days/Case'] = monthly_summary_statuscounts.apply(lambda x: (  ftedays_status_change_count * x['Group Status Changes/Case']  ) , axis=1)
#monthly_summary_statuscounts['Avg FTE Days/Case'] = monthly_summary_statuscounts.apply(lambda x: (  ftedays_status_change_count * x['Avg Yearly Status Changes/Case']  ) , axis=1)
monthly_summary_statuscounts['Group FTE Days'] = monthly_summary_statuscounts.apply(lambda x: (  ftedays_status_change_count * x['Total Status Change Count']  ) , axis=1)

#Total Status Changes in Group

#monthly_summary_statuscounts

#### Status Changes/Case by Case Type

The Figure 3. below displays the average Status Changes/Case by Case Type, expressed in Status Changes/Case.
The overall yearly average Status Changes/Case is also indicated by the Red line across the chart.

Tooltip Data
* Total Cases in Year
* Total Status Changes in Group
* Group Status Changes/Case


#### Average FTE Days by Case Origin



Tooltip Data
* Total Cases in Year
* Total Status Changes in Group
* Group Status Changes/Case


In [ ]:
#hide
base = alt.Chart(monthly_summary_statuscounts)

if 0:
    bar1 = base.mark_bar( opacity=.5).encode(
        x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='Case Origin') ),
        y=alt.Y("Group Status Changes/Case:Q", axis=alt.Axis(title='Status Changes per Case') ),    
        tooltip=['Total Status Change Count', 'Case Count', 'Group Status Changes/Case:Q']
    )
    rule1 = base.mark_rule(color='red').encode(
        y='Total Status Changes In Month:Q',
        tooltip = ['Total Status Changes In Month', 'Avg Yearly Status Changes/Case:Q']
    )
    bar2 = base.mark_bar(opacity=.5).encode(
        x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='Case Origin') ),
        y=alt.Y("Group FTE Days/Case:Q", axis=alt.Axis(title='FTE Days per Case') ),
        tooltip=['Total Cases in Year:Q', 'Total Status Change Count:Q','Group Status Changes/Case:Q']
    )
    rule2 = base.mark_rule(color='red').encode(
        y='Avg FTE Days/Case:Q'
    )
    bar3 = base.mark_bar(opacity=.5).encode(
        x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='') ),
        y=alt.Y("mean(Case Count):Q", axis=alt.Axis(title='Case Count') ),
        tooltip=['% Group Cases:Q','Case Count','Total Status Change Count','Group FTE Days','Group Status Changes/Case:Q']
    )
    rule_mean3 = base.mark_rule(color='red').encode(
        x=alt.X('mean(Status Count):Q', axis=None),
        size=alt.value(5),
        tooltip=['mean(Status Count)', 'sum(DL)']
    )
    line3 = base.mark_line(color='red',point=True).encode(
        x=alt.X("ORIGIN_DSC:N", axis=None ),
        y=alt.Y("Group FTE Days:Q", axis=alt.Axis(title='FTE Days') ),
        tooltip=['Group FTE Days']
    )
    (bar1 + rule1) | (bar2 + rule2) | alt.layer(bar3, line3).resolve_scale( y = 'independent').properties(width=400)

else:
    bar = base.mark_line( ).encode(
        x=alt.X("CASE_OPENED_DT:T", axis=alt.Axis(title='Month') ),
        y=alt.Y("Group Status Changes/Case:Q", axis=alt.Axis(title='Status Changes per Case') ),    
        tooltip=['Total Status Change Count', 'Case Count', 'Group Status Changes/Case:Q'],
        column='ORIGIN_DSC'
    )

bar

alt.Chart(...)

In [ ]:
#hide

base = alt.Chart(monthly_summary_statuscounts)

bar1 = base.mark_bar( opacity=.5).encode(
    y=alt.Y("ORIGIN_DSC:N", axis=alt.Axis(title='Case Origin') ),
    x=alt.X("Group Status Changes/Case:Q", axis=alt.Axis(title='Status Changes per Case') ),    
    tooltip=['Total Status Change Count', 'Case Count', 'Group Status Changes/Case:Q'],
    #row='CASE_OPENED_DT:T'
)
rule1 = base.mark_rule(color='red').encode(
    x='Monthly Status Changes/Case:Q',
    tooltip = 'Avg Yearly Status Changes/Case:Q'
)
# bar2 = base.mark_bar(opacity=.5).encode(
#     x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='Case Origin') ),
#     y=alt.Y("Group FTE Days/Case:Q", axis=alt.Axis(title='FTE Days per Case') ),
#     tooltip=['Total Cases in Year:Q', 'Total Status Change Count:Q','Group Status Changes/Case:Q']
# )
# rule2 = base.mark_rule(color='red').encode(
#     y='Avg FTE Days/Case:Q'
# )
# bar3 = base.mark_bar(opacity=.5).encode(
#     x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='') ),
#     y=alt.Y("mean(Case Count):Q", axis=alt.Axis(title='Case Count') ),
#     tooltip=['% Group Cases:Q','Case Count','Total Status Change Count','Group FTE Days','Group Status Changes/Case:Q']
# )
# rule_mean3 = base.mark_rule(color='red').encode(
#     x=alt.X('mean(Status Count):Q', axis=None),
#     size=alt.value(5),
#     tooltip=['mean(Status Count)', 'sum(DL)']
# )
# line3 = base.mark_line(color='red',point=True).encode(
#     x=alt.X("ORIGIN_DSC:N", axis=None ),
#     y=alt.Y("Group FTE Days:Q", axis=alt.Axis(title='FTE Days') ),
#     tooltip=['Group FTE Days']
# )
# (bar1 + rule1) | (bar2 + rule2) | alt.layer(bar3, line3).resolve_scale( y = 'independent').properties(width=400)
(bar1 + rule1).facet(row='CASE_OPENED_DT:T')


alt.FacetChart(...)

In [ ]:
#hide

#base = alt.Chart(case_summary_data)
base = alt.Chart(monthly_summary_statuscounts)




bar1 = base.mark_bar( opacity=.5).encode(
    x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='Case Origin') ),
    #y=alt.Y("FTE Day/Case:Q", axis=alt.Axis(title='FTE Day per Case') ),
    y=alt.Y("Group Status Changes/Case:Q", axis=alt.Axis(title='Status Changes per Case') ),
#   row = 'CASE_OPENED_DT',
    tooltip=['Total Status Change Count', 'Case Count', 'Group Status Changes/Case:Q']

)
rule1 = base.mark_rule(color='red').encode(
#    y='Avg Yearly Status Changes/Case:Q',
    y='Monthly Status Changes/Case:Q', #Group Status Changes/Case
    tooltip = 'Avg Yearly Status Changes/Case:Q',
#    row = 'CASE_OPENED_DT'
)


bar2 = base.mark_bar(opacity=.5).encode(
    x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='Case Origin') ),
    #y=alt.Y("FTE Day/Case:Q", axis=alt.Axis(title='FTE Day per Case') ),
    y=alt.Y("Group FTE Days/Case:Q", axis=alt.Axis(title='FTE Days per Case') ),
    
    tooltip=['Total Cases in Year:Q', 'Total Status Change Count:Q','Group Status Changes/Case:Q']

)

rule2 = base.mark_rule(color='red').encode(
    y='Avg FTE Days/Case:Q'
)

bar3 = base.mark_bar(opacity=.5).encode(
    x=alt.X("ORIGIN_DSC:N", axis=alt.Axis(title='') ),
    #y=alt.Y("sum(Cases):Q", axis=alt.Axis(title='Case Count') ),
    #y=alt.Y("Total Cases in Group:Q", axis=alt.Axis(title='Case Count') ),
    y=alt.Y("mean(Case Count):Q", axis=alt.Axis(title='Case Count') ),
    
#    tooltip=['% Group Cases:Q','Total Cases in Group','Total Status Change Count','Group FTE Days','Group Status Changes/Case:Q']
    tooltip=['% Group Cases:Q','Case Count','Total Status Change Count','Group FTE Days','Group Status Changes/Case:Q']

)

rule_mean3 = base.mark_rule(color='red').encode(
    x=alt.X('mean(Status Count):Q', axis=None),
    size=alt.value(5),
    tooltip=['mean(Status Count)', 'sum(DL)']
)

line3 = base.mark_line(color='red',point=True).encode(
    x=alt.X("ORIGIN_DSC:N", axis=None ),
    y=alt.Y("Group FTE Days:Q", axis=alt.Axis(title='FTE Days') ),
    #size=alt.value(1),
    tooltip=['Group FTE Days']
)

#bar + line
# alt.layer(bar, line3).resolve_scale(
#     y = 'independent'
# ).properties(width = 700, height=400)


#(bar1 + rule1) | (bar2 + rule2) | alt.layer(bar3, line3).resolve_scale( y = 'independent').properties(width=400)
#alt.layer(bar3, line3).resolve_scale( y = 'independent').properties(width=400)

(bar1 + rule1).facet( row = 'CASE_OPENED_DT')

alt.FacetChart(...)

Figure 7. 
a) Status Changes per Case type 
b) FTE Days per case, by Case Type 
c) Case Count and FTE Days, by Case Type

## Case Durations by Case Origin

In [ ]:
#hide
base = alt.Chart(case_agecategory_summary_data)


bar = base.mark_bar().encode(
    x=alt.X("Case Origin:N", axis=alt.Axis(title='Origin') ),
    #y=alt.Y("sum(Cases):Q", axis=alt.Axis(title='Case Count') ),
    y=alt.Y("Case Count:Q", axis=alt.Axis(title='Case Count') ),
#    tooltip=['% Group Cases:Q','Total Cases in Group','Total Status Changes in Group','Group FTE Days','Group Status Changes/Case:Q']
)

bar2 = base.mark_bar().encode(
    x=alt.X("Case Origin:N", axis=alt.Axis(title='Origin') ),
    #y=alt.Y("sum(Cases):Q", axis=alt.Axis(title='Case Count') ),
    y=alt.Y("Status Changes per Case:Q", axis=alt.Axis(title='Status Changes per Case') ),
#    tooltip=['% Group Cases:Q','Total Cases in Group','Total Status Changes in Group','Group FTE Days','Group Status Changes/Case:Q']
)
bar3 = base.mark_bar().encode(
    x=alt.X("Case Origin:N", axis=alt.Axis(title='Origin') ),
    #y=alt.Y("sum(Cases):Q", axis=alt.Axis(title='Case Count') ),
    y=alt.Y("Average Case Length:Q", axis=alt.Axis(title='Average Case Length (Days)') ),
#    tooltip=['% Group Cases:Q','Total Cases in Group','Total Status Changes in Group','Group FTE Days','Group Status Changes/Case:Q']
)
bar4 = base.mark_bar().encode(
    x=alt.X("Case Origin:N", axis=alt.Axis(title='Origin') ),
    #y=alt.Y("sum(Cases):Q", axis=alt.Axis(title='Case Count') ),
    y=alt.Y("Average Turnaround Time:Q", axis=alt.Axis(title='Average Turnaround Time (Days)') ),
#    color='Age Category'
#    tooltip=['% Group Cases:Q','Total Cases in Group','Total Status Changes in Group','Group FTE Days','Group Status Changes/Case:Q']
)

rule_mean = base.mark_rule(color='red').encode(
    x=alt.X('mean(Status Count):Q', axis=None),
    size=alt.value(5),
    tooltip=['mean(Status Count)', 'sum(DL)']
)

line = base.mark_line(color='red',point=True).encode(
    x=alt.X("Case Origin:N", axis=None ),
    y=alt.Y("Group FTE Days:Q", axis=alt.Axis(title='FTE Days') ),
    #size=alt.value(1),
    tooltip=['Group FTE Days']
)

bar | bar2 | bar3 | bar4
# alt.layer(bar, line).resolve_scale(
#     y = 'independent'
# ).properties(width = 700, height=400)


# alt.layer(bar).facet(
#     'Case Type:N',
#     columns=2
# )

alt.HConcatChart(...)

Figure 8. 
a) Case Count
b) Status Changes/Case
c) Average Case Length
d) Average Turnaround Time

In [ ]:
#hide

# #turnaround_time_days, case_length_days

# points = alt.Chart(cases2018_df).mark_point(
#     filled=True,
#     color='black'
# ).encode(
#     x=alt.X('mean(case_length_days)', title='Case Length'),
#     y=alt.Y(
#         'CASE_DSC',
#          sort=alt.EncodingSortField(
#              field='case_length_days',
#              op='mean',
#              order='descending'
#          )
#     )
# ).properties(
#     width=400,
#     height=250
# )

# error_bars = points.mark_rule().encode(
#     x='ci0(case_length_days)',
#     x2='ci1(case_length_days)',
# )

# points + error_bars

In [ ]:
#hide

# aggregation = {
#     'mean_temp': ('case_length_days','mean'),
#     'bin_min': ('case_length_days','min'),
#     'bin_max': ('case_length_days','max')

# }

# #d.groupby(['CASE_DSC']).count()

# groupby_cols = ['CASE_DSC']

# bins = pd.cut(d["case_length_days"], bins=7, labels=("very low", "low", "lower med", "med", "upper med", "high", "very high"))
# #groupby_cols += bins
# #print(bins)

# test_summary = d.groupby(['CASE_DSC', bins]).agg(** aggregation).reset_index()

# #fillna(0)

# test_summary['mean_temp'] = test_summary[['mean_temp']].fillna(0)
# test_summary['bin_min'] = test_summary[['bin_min']].fillna(0)
# test_summary['bin_max'] = test_summary[['bin_max']].fillna(0)
# #test_summary.fillna(0)
# #bins = pd.cut(test_summary["mean_temp"], bins=3, labels=("cool", "warm", "hot"))
# #df[["rel_hum", "abs_hum"]].groupby(bins).agg(["mean", "median"])

# # bla = test_summary[['mean_temp']].groupby(['CASE_DSC', bins]).agg('count')
# # print(bla)

# #test_summary

In [ ]:
#hide

# #'GENERAL_STATUS', 'Is ERA Dsc'

# cases_df.groupby(['GENERAL_STATUS', 'Is ERA Dsc']).count()

# cases_df[cases_df['GENERAL_STATUS'].isna()][['STATUS_DSC', 'DECISION_DSC']]

In [ ]:
#hide

# single_status = cases2018_df[cases2018_df['STATUS_COUNT'] == 1][['SECOND_CASE_STATUS_CD','STATUS_HISTORY_y','DECISION_DSC']]

# #cases2018_df.columns
# single_status['SECOND_CASE_STATUS_CD'].value_counts()

In [ ]:
#hide

aggregation = {
    'Case Count': ('STATUS_COUNT','size'),
    'Total Status Change Count': ('STATUS_COUNT','sum')
}
case_summary_data_by_status_type = cases2018_df.groupby(['Second Status Desc']) \
.agg(** aggregation).reset_index()

print(f"Total Case Count: {case_summary_data_by_status_type['Case Count'].sum():,.0f}" )
print(f"Total Status Change Count: {case_summary_data_by_status_type['Total Status Change Count'].sum():,.0f}" )

bar1 = alt.Chart(case_summary_data_by_status_type).mark_bar().encode(
    alt.Y('sum(Case Count):Q', title="Total Cases"),
    x= 'Second Status Desc',
    #alt.X('Age Category'),
    #x = alt.X('Case Count', aggregate='sum', type='quantitative'),
    tooltip=['sum(Case Count):Q'],
    #row='SECOND_CASE_STATUS_CD:N'
)

bar1

# bar2 = alt.Chart(case_summary_data_by_status_type).mark_bar().encode(
#     #alt.X(alt.X('Age Category'),
#     #x = alt.X('Total Status Change Count', aggregate='sum', type='quantitative', title="Total Status Changes"),
#     x = alt.X('sum(Total Status Change Count):Q', title="Total Status Changes"),
#     tooltip=['sum(Total Status Change Count):Q'],
#     color='Age Category'
# )
# bar1 & bar2

Total Case Count: 10,745
Total Status Change Count: 40,054


alt.Chart(...)

In [ ]:
#hide_input


selection = alt.selection_multi(fields=['Origin & Decision']) #, bind='legend')

base = alt.Chart(daily_case_summary_data)

xscale = alt.Scale(domain=(1.0, 1000))
yscale = alt.Scale(domain=(1.9, 4.55))

area_args = {'opacity': .05, 'interpolate': 'step'}
#area_args = {'interpolate': 'step'}

points = base.mark_point().encode(
    alt.X('Average Case Length in Days:Q'), #,  scale=xscale),
    alt.Y('STATUS_COUNT', type='quantitative'), #, scale=yscale),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.05)),
    #color='Type & Origin Desc',
    color=alt.Color('Origin & Decision',
            sort=alt.EncodingSortField('count(STATUS_COUNT)',  order='descending')),
    tooltip='Cases'
#)
# .add_selection(
#     selection
).interactive()

bottom_hist = base.mark_bar().encode(
    alt.X('Origin & Decision:N',
          # when using bins, the axis scale is set through
          # the bin extent, so we do not specify the scale here
          # (which would be ignored anyway)
          #bin=alt.Bin(maxbins=20 ), # , extent=xscale.domain),
          stack=None,
          title='', sort='-y',
         ),
    alt.Y('sum(Cases)', stack=None, title=''),
    alt.Color('Origin & Decision:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01)),


).properties(height=60)

top_hist = base.mark_area(**area_args).encode(
    alt.X('Average Case Length in Days:Q',
          # when using bins, the axis scale is set through
          # the bin extent, so we do not specify the scale here
          # (which would be ignored anyway)
          bin=alt.Bin(maxbins=20 ), # , extent=xscale.domain),
          stack=None,
          title=''
         ),
    alt.Y('sum(Cases)', stack=None, title=''),
    alt.Color('Origin & Decision:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01))

).properties(height=60)

right_hist = base.mark_area(**area_args).encode(
    alt.Y('STATUS_COUNT:Q',
          bin=alt.Bin(maxbins=20), #, extent=yscale.domain),
          stack=None,
          title='',
         ),
    alt.X('sum(Cases)', stack=None, title=''),
    alt.Color('Origin & Decision:N'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.01))
).properties(width=60)

color = alt.condition(selection,
                      alt.Color('Origin & Decision:O', legend=None, 
                      scale=alt.Scale(scheme='category10')),
                      alt.value('lightgray'))

legend = base.mark_point(filled=True, size=200
).encode(
    y=alt.Y('Origin & Decision:O', axis=alt.Axis(orient='right')),
    color=color,
    size='sum(Cases)'
).add_selection(
selection
)

legend | (top_hist & (points | right_hist) & bottom_hist ).add_selection(
    selection
)
    

alt.HConcatChart(...)

In [ ]:
#hide

#monthly_summary_statuscounts[monthly_summary_statuscounts['ORIGIN_DSC'] == 'DMER - DRIVER MEDICAL EXAMINATION REPORT']


In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()


Converted 00_core.ipynb.
Converted 01_core.baseline.ipynb.
Converted 10. Cost Analysis Baseline - 2018 Over 80 Cases by Case Origin.ipynb.
Converted index.ipynb.
